In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GlobalAveragePooling2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam

import matplotlib.pyplot as plt

import numpy as np
from scipy.misc import imread
from scipy.misc import imresize
from scipy.misc import imsave
import glob
import random

from keras.preprocessing.image import ImageDataGenerator
from keras import applications
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical
#from keras.applications.resnet50 import preprocess_input

from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
#from keras.applications.vgg19 import preprocess_input as preprocess_vgg
from keras.applications.inception_v3 import preprocess_input as inception_preprocess

#from keras.applications.inception_resnet_v2 import preprocess_input

import numpy as np

import keras
import copy
import cv2

from keras.models import *
from keras.layers import *
from keras.optimizers import *

import numpy as np 
import os
import numpy as np
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras

Using TensorFlow backend.


In [2]:
path = '../data/'
savepath = '../data/'
#images = glob.glob(path+'new_data/*.npy')

In [15]:
tile_sizes = np.loadtxt(path+'tile_sizes.txt', dtype='int')
tile_sizes = tile_sizes[350:400]
images_sampled = {}
for tile in tile_sizes:
    for i in range(1, 201):
        images_sampled.setdefault(tile[0]*30+i, []).append(tile[1])
print(len(images_sampled.keys()))

200


In [31]:
def myGenerator(batch_size):
    s = 640
    while True:
        index_list = random.sample(images_sampled.keys(), batch_size)
        alldata_x = []
        alldata_y = []
        for i in index_list:
            frame = path+'sources/nparrs_384/frame'+str(i)+'.npy'
            frame = np.load(frame)
            tile_index = random.choice(images_sampled[i])
            
            tmp = imresize(frame[tile_index], (s, 480))
            temp  = imresize(frame[tile_index], (48, 48))
            temp  = imresize(temp, (s, 480))
            
            alldata_x.append(temp)
            #alldata_y.append(frame[tile_index])
            alldata_y.append(tmp)
        
        alldata_x = np.array(alldata_x)
        alldata_y = np.array(alldata_y)
        
        alldata_y = (alldata_y.astype(np.float32) - 127.5) / 127.5
        alldata_x = alldata_x.astype(np.float32)/255.0
        
        yield alldata_x, alldata_y

#x = myGenerator(10)
#xtrain, ytrain = next(x)
#print('xtrain shape:',xtrain.shape)
#print('ytrain shape:',ytrain.shape)

In [32]:
class CGAN():
    def __init__(self):
        # Input shape
        s = 640
        self.img_rows = s
        self.img_cols = 480
        self.channels = 3
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        #self.discriminator = self.build_discriminator()
        #self.discriminator.compile(loss=['mse'],
        #    optimizer=optimizer,
        #    metrics=['accuracy'])
        
        #print(self.discriminator.summary())
        
        # Build the generator
        self.generator = self.build_generator()
        
        print(self.generator.summary())
        
        noise = Input(shape=(self.img_rows, self.img_cols, 3))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        #self.discriminator.trainable = False

        # The discriminator takes generated image as input and determines validity
        # and the label of that image
        #valid = self.discriminator(img)
        
        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        #self.combined = Model(noise, [img, valid])
        #self.combined.compile(loss=['mse', 'mse'],
        #    loss_weights=[0.9, 0.1],
        #    optimizer=optimizer)

    def build_generator(self):
        filter_size = 8
        input_size = (self.img_rows, self.img_cols,3)
        inputs = Input(input_size)

        conv1 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(inputs)
        conv1 = LeakyReLU()(conv1)
        conv1 = BatchNormalization(momentum=0.8)(conv1)

        conv2 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv1)
        conv2 = LeakyReLU()(conv2)
        conv2 = BatchNormalization(momentum=0.8)(conv2)

        conv3 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv2)
        conv3 = LeakyReLU()(conv3)
        conv3 = BatchNormalization(momentum=0.8)(conv3)

        concat1 = add([conv1, conv3])

        conv4 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(concat1)
        conv4 = LeakyReLU()(conv4)
        conv4 = BatchNormalization(momentum=0.8)(conv4)

        conv5 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv4)
        conv5 = LeakyReLU()(conv5)
        conv5 = BatchNormalization(momentum=0.8)(conv5)

        concat2 = add([conv5, concat1])

        conv6 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(concat2)
        conv6 = LeakyReLU()(conv6)
        conv6 = BatchNormalization(momentum=0.8)(conv6)

        conv7 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv6)
        conv7 = LeakyReLU()(conv7)
        conv7 = BatchNormalization(momentum=0.8)(conv7)

        concat3 = add([conv7, concat2])

        conv8 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(concat3)
        conv8 = LeakyReLU()(conv8)
        conv8 = BatchNormalization(momentum=0.8)(conv8)

        conv9 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv8)
        conv9 = LeakyReLU()(conv9)
        conv9 = BatchNormalization(momentum=0.8)(conv9)
        

        conv10 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv9)
        conv10 = LeakyReLU()(conv10)
        conv10 = BatchNormalization(momentum=0.8)(conv10)

        concat4 = add([conv10, conv1])
        
        conv11 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(concat4)
        conv11 = LeakyReLU()(conv11)
        conv11 = BatchNormalization(momentum=0.8)(conv11)
        
        conv12 = Conv2D(filter_size, 3, padding = 'same', kernel_initializer = 'he_normal')(conv11)
        conv12 = PReLU()(conv12)
        conv12 = BatchNormalization(momentum=0.8)(conv12)


        out = Conv2D(3, 3, padding = 'same', kernel_initializer = 'he_normal')(conv12)
        out = LeakyReLU()(out)
        out = BatchNormalization(momentum=0.8)(out)


        model = Model(input = inputs, output = out)
        return model
    
    def build_autoencoder(self):
        self.generator.compile(loss=['mse'],optimizer=self.optimizer)
    
    def train_generator_autoencoder(self, epochs, batch_size=128, sample_interval=10):
        for epoch in range(epochs):
            X_train, y_train = next(myGenerator(batch_size))
            g_loss = self.generator.train_on_batch(X_train, y_train)
            print ("Epoch ", epoch, " G loss ", g_loss)
            #if epoch % sample_interval == 0:
                #self.sample_images(epoch)
                #self.generator.save_weights(savepath+'Per-segment-models/8/generator_weights_'+str(epoch)+'.h5')

    def sample_images(self, epoch):
        r, c             = 1, 10
        X_train, y_train = next(myGenerator(10))
        gen_imgs         = self.generator.predict(X_train)
        
        # Rescale images 0 - 1
        temp     = (0.5 * gen_imgs + 0.5)*255
        gen_imgs = temp.astype(int)
        y_train = (0.5 * y_train + 0.5)*255
        y_train = y_train.astype(int)
        X_train = X_train*255
        X_train = X_train.astype(int)
        
        combined = np.array([gen_imgs[0], gen_imgs[1], gen_imgs[2], gen_imgs[3], gen_imgs[4], gen_imgs[5], gen_imgs[6], gen_imgs[7], gen_imgs[8], gen_imgs[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+".png", combined)
        
        combined = np.array([y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], y_train[5], y_train[6], y_train[7], y_train[8], y_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_real.png", combined)
        
        combined = np.array([X_train[0], X_train[1], X_train[2], X_train[3], X_train[4], X_train[5], X_train[6], X_train[7], X_train[8], X_train[9]])
        combined = np.hstack(combined.reshape(10, 384,384, 3))
        imsave(savepath+"images/"+str(epoch)+"_lowres.png", combined)

In [33]:
cgan = CGAN()
cgan.build_autoencoder()
cgan.train_generator_autoencoder(25, 2, 10)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:109: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ba..., inputs=Tensor("in...)`


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_13 (InputLayer)           (None, 640, 480, 3)  0                                            
__________________________________________________________________________________________________
conv2d_79 (Conv2D)              (None, 640, 480, 8)  224         input_13[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_73 (LeakyReLU)      (None, 640, 480, 8)  0           conv2d_79[0][0]                  
__________________________________________________________________________________________________
batch_normalization_79 (BatchNo (None, 640, 480, 8)  32          leaky_re_lu_73[0][0]             
__________________________________________________________________________________________________
conv2d_80 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


Epoch  0  G loss  2.2478147
Epoch  1  G loss  2.3788047
Epoch  2  G loss  1.7379737
Epoch  3  G loss  2.1157131
Epoch  4  G loss  1.837394
Epoch  5  G loss  1.5790558
Epoch  6  G loss  1.870225
Epoch  7  G loss  1.4191728
Epoch  8  G loss  1.8853705
Epoch  9  G loss  1.3648465
Epoch  10  G loss  1.9276135
Epoch  11  G loss  1.2842991
Epoch  12  G loss  1.5337355
Epoch  13  G loss  1.5118074
Epoch  14  G loss  1.3931231
Epoch  15  G loss  1.7390432
Epoch  16  G loss  1.4629221
Epoch  17  G loss  1.3320756
Epoch  18  G loss  1.3877605
Epoch  19  G loss  1.1749347
Epoch  20  G loss  1.4510566
Epoch  21  G loss  1.3590536
Epoch  22  G loss  1.3707373
Epoch  23  G loss  1.7077756
Epoch  24  G loss  1.2668923


In [34]:
import time
for i in range(30):
    x = myGenerator(30)
    xtest, ytest = next(x)
    a = time.time()
    pred = cgan.generator.predict(xtest)
    b = time.time()
    print(b-a)
    pred = pred*255
    pred = pred.astype(int)
    #plt.imshow(pred[0])
    #plt.show()
    ytest = ytest*127.5+127.5
    ytest = ytest.astype(int)
    #plt.imshow(ytest[0])
    #plt.show()
    #imsave(path+'test/frame_pred'+str(i)+'.png', pred[0])
    #imsave(path+'test/frame_real'+str(i)+'.png', ytest[0])

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  if sys.path[0] == '':
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  del sys.path[0]
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  


4.823161840438843
0.9678699970245361
0.9798541069030762
0.9841511249542236
0.9934227466583252
0.971282958984375
0.9897301197052002
0.9745113849639893
0.9787354469299316
1.0341806411743164
0.9499044418334961
0.9322893619537354
0.9754765033721924
0.9426059722900391
0.9431202411651611
0.9533538818359375
0.9376301765441895
0.953239917755127
0.9701035022735596
0.9451735019683838
0.9389424324035645
0.947298526763916
0.9366376399993896
0.9384880065917969
0.9442243576049805
0.9448287487030029
0.9380161762237549
0.9380192756652832
0.9408519268035889
0.9456193447113037


In [ ]:
import numpy 
import math
import cv2

def psnr(img1, img2):
    mse = numpy.mean( (img1 - img2) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 255.0
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

In [ ]:
data = []
for i in range(100):
    img1 = cv2.imread(path+'test/frame_real'+str(i)+'.png')
    img2 = cv2.imread(path+'test/frame_pred'+str(i)+'.png')
    d = psnr(img1, img2)
    data.append(d)
    print(d)
print('PSNR:', np.mean(data))